In [ ]:
import os
import scipy.io as sio
import numpy as np

def load_eeg_data(directory):
    data = []
    for filename in os.listdir(directory):
        if filename.endswith('.mat'):
            filepath = os.path.join(directory, filename)
            mat_data = sio.loadmat(filepath)
            data.append(mat_data)
    return data

eeg_data_directory = 'D:\PyCharm\EEG_ID\pre_data'  # 修改为MAT文件所在的目录
eeg_data = load_eeg_data(eeg_data_directory)
